In [ ]:
#!/usr/bin/env python3
"""
Trains a convolutional neural network to classify the CIFAR 10 dataset:
"""
# %tensorflow_version 1.x
import tensorflow.keras as K
import tensorflow as tf


def preprocess_data(X, Y):
    """
    a function that trains a convolutional neural network to classify the
    CIFAR 10 dataset
    :param X: X is a numpy.ndarray of shape (m, 32, 32, 3) containing the
    CIFAR 10 data, where m is the number of data points
    :param Y: Y is a numpy.ndarray of shape (m,) containing the CIFAR 10
    labels for X
    :return: X_p, Y_p
        X_p is a numpy.ndarray containing the preprocessed X
        Y_p is a numpy.ndarray containing the preprocessed Y
    """
    X_p = K.applications.resnet50.preprocess_input(X)
    Y_p = K.utils.to_categorical(Y, 10)
    return X_p, Y_p


if __name__ == "__main__":
    (x_train, y_train), (x_test, y_test) = K.datasets.cifar10.load_data()
    print((x_train.shape, y_train.shape))
    x_train, y_train = preprocess_data(x_train, y_train)
    x_test, y_test = preprocess_data(x_test, y_test)
    print((x_train.shape, y_train.shape))

    input_t = K.Input(shape=(32, 32, 3))
    res_model = K.applications.ResNet50(include_top=False,
                                        weights="imagenet",
                                        input_tensor=input_t)
    
    
    for layer in res_model.layers[:143]:
        layer.trainable = False

#     for i, layer in enumerate(res_model.layers):
#         print(i, layer.name, "-", layer.trainable)

    to_res = (32, 32)
    model = K.models.Sequential()
    model.add(K.layers.Lambda(lambda image: tf.image.resize(image, to_res))) 
    model.add(res_model)
    model.add(K.layers.Flatten())
    model.add(K.layers.BatchNormalization())
    model.add(K.layers.Dense(256, activation='relu'))
    model.add(K.layers.Dropout(0.5))
    model.add(K.layers.BatchNormalization())
    model.add(K.layers.Dense(128, activation='relu'))
    model.add(K.layers.Dropout(0.5))
    model.add(K.layers.BatchNormalization())
    model.add(K.layers.Dense(64, activation='relu'))
    model.add(K.layers.Dropout(0.5))
    model.add(K.layers.BatchNormalization())
    model.add(K.layers.Dense(10, activation='softmax'))

    

    check_point = K.callbacks.ModelCheckpoint(filepath="cifar10.h5",
                                              monitor="val_acc",
                                              mode="max",
                                              save_best_only=True,
                                              )

    model.compile(loss='categorical_crossentropy',
                  optimizer=K.optimizers.RMSprop(lr=2e-5),
                  metrics=['accuracy'])
    history = model.fit(x_train, y_train, batch_size=32, epochs=10, verbose=1,
                        validation_data=(x_test, y_test),
                        callbacks=[check_point])
#     model.summary()
    model.save("cifar10.h5")

((50000, 32, 32, 3), (50000, 1))
((50000, 32, 32, 3), (50000, 10))
Epoch 1/10
1563/1563 [==============================] - 79s 46ms/step - loss: 2.9235 - accuracy: 0.1082 - val_loss: 2.1092 - val_accuracy: 0.2588
Epoch 2/10
1563/1563 [==============================] - 72s 46ms/step - loss: 2.5864 - accuracy: 0.1606 - val_loss: 1.8701 - val_accuracy: 0.3871
Epoch 3/10
1563/1563 [==============================] - 72s 46ms/step - loss: 2.2976 - accuracy: 0.2290 - val_loss: 1.6770 - val_accuracy: 0.4593
Epoch 4/10
1563/1563 [==============================] - 74s 47ms/step - loss: 2.0635 - accuracy: 0.2936 - val_loss: 1.5157 - val_accuracy: 0.5222
Epoch 5/10
1563/1563 [==============================] - 73s 47ms/step - loss: 1.8756 - accuracy: 0.3550 - val_loss: 1.3970 - val_accuracy: 0.5629
Epoch 6/10
1437/1563 [==========================>...] - ETA: 5s - loss: 1.7314 - accuracy: 0.4137

In [ ]:
# preprocess_data = __import__('0-transfer').preprocess_data

# fix issue with saving keras applications
K.learning_phase = K.backend.learning_phase

_, (X, Y) = K.datasets.cifar10.load_data()
X_p, Y_p = preprocess_data(X, Y)
model = K.models.load_model('cifar10.h5')
model.evaluate(X_p, Y_p, batch_size=128, verbose=1)

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'bo', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'ro', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()